In [1]:
#if haven't installed ee package, please run the command in next line first
#conda install -c conda-forge earthengine-api

#keep up to date
#conda update -c conda-forge earthengine-api

#install folium package for interactive map
#conda install folium

import ee
import folium
from folium import plugins

#need to generate token once you authorised
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AdQt8qgZVICs7fDgi2ApLP1wIE--FTUDLPyLOD_2TFMsOVhA8OibZ0Sgo0g

Successfully saved authorization token.


In [29]:
geometry = ee.FeatureCollection("projects/ee-gupei/assets/Final_data_9k")

In [3]:
#sentinel 2A
Sentinel2A = ee.ImageCollection("COPERNICUS/S2_SR");

In [31]:
#set the bands
BANDS = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12'];
palette = [
  '0000FF', #Water
  '008000', #Veg
  'A52A2A'  #Land
]

In [5]:
#filter by date range, and Bands
ic = Sentinel2A.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
ic = ic.select(BANDS).filterDate('2016-07-01', '2020-12-01').median();

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split

In [14]:
scenes = ic.getInfo()
[print(scene) for scene in scenes]

type
bands


[None, None]

In [19]:
randomforest_classifier = ee.FeatureCollection("projects/ee-gupei/assets/RandomForest-new");
classifier_string = randomforest_classifier.first().get('classifier');
classifier = ee.Classifier.smileRandomForest(classifier_string);

In [20]:
classified = ic.classify(classifier);

In [26]:
lat = -37.8
lon = 145.2
Map = folium.Map(location=[lat, lon], zoom_start=10)

def add_ee_layer(self, ee_object, vis_params, name):
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)
        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))
    
folium.Map.add_ee_layer = add_ee_layer


In [33]:
vis_params = {'palette': palette, 'min': 0, 'max': 2};
name = "Random Forest";
Map.add_ee_layer(geometry, vis_params, name)
Map